In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [139]:
client_data = "../data/raw/df_final_demo.csv"
experiment_data = "../data/raw/df_final_experiment_clients.csv"
web_data1 = "../data/raw/df_final_web_data_pt_1.csv"
web_data2 = "../data/raw/df_final_web_data_pt_2.csv"

In [140]:
client_df = pd.read_csv(client_data)
experiment_df = pd.read_csv(experiment_data)
web1_df = pd.read_csv(web_data1)
web2_df = pd.read_csv(web_data2)

In [141]:
client_df.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


In [142]:
client_df.shape

(70609, 9)

In [143]:
web1_df.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,17/04/17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,17/04/17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,17/04/17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,17/04/17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,17/04/17 15:18:04


In [144]:
web2_df.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,06/06/17 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,01/06/17 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,01/06/17 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,01/06/17 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,01/06/17 11:57:58


In [145]:
web_df = pd.concat([web1_df, web2_df], axis=0, ignore_index=True)

In [146]:
web_df.shape

(755405, 5)

In [147]:
web_df.tail()

,client_id,visitor_id,visit_id,process_step,date_time
755400,9668240,388766751_9038881013,922267647_3096648104_968866,start,24/05/17 18:46:10
755401,9668240,388766751_9038881013,922267647_3096648104_968866,start,24/05/17 18:45:29
755402,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,24/05/17 18:44:51
755403,9668240,388766751_9038881013,922267647_3096648104_968866,start,24/05/17 18:44:34
755404,674799,947159805_81558194550,86152093_47511127657_716022,start,03/06/17 12:17:09


In [148]:
display(web1_df.shape)
display(web2_df.shape)

(343141, 5)

(412264, 5)

In [149]:
merged_client_df = pd.merge(client_df, experiment_df, on='client_id', how='outer')

In [150]:
experiment_df.head()

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control


In [151]:
merged_client_df['Variation'].value_counts()

Variation
Test       26968
Control    23532
Name: count, dtype: int64

In [152]:
merged_client_df.head(20)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation
0,169,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0,NaN
1,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test
2,647,12.0,151.0,57.5,M,2.0,30525.80,0.0,4.0,Test
3,722,11.0,143.0,59.5,F,2.0,22466.17,1.0,1.0,NaN
4,934,9.0,109.0,51.0,F,2.0,32522.88,0.0,3.0,Test
5,1028,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control
6,1104,5.0,66.0,48.0,U,3.0,154643.94,6.0,9.0,Control
7,1186,8.0,99.0,22.0,U,2.0,31662.52,0.0,3.0,Control
8,1195,21.0,262.0,54.5,M,2.0,28457.96,2.0,5.0,Control
9,1197,7.0,85.0,28.5,U,5.0,177062.14,5.0,8.0,Control


In [153]:
merged_client_df[merged_client_df['Variation'].isnull()].shape

(20109, 10)

In [154]:
merged_client_df.shape

(70609, 10)

In [155]:
merged_client_df[merged_client_df['gendr'].isnull()].head(20)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation
2486,355337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Control
7251,1037867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
8525,1227228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
11296,1618749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15523,2222915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
34329,4876926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
36220,5144725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
37164,5277910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
42470,6021001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52406,7402828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Control


In [156]:
merged_client_df.isnull().sum()

client_id               0
clnt_tenure_yr         14
clnt_tenure_mnth       14
clnt_age               15
gendr                  14
num_accts              14
bal                    14
calls_6_mnth           14
logons_6_mnth          14
Variation           20109
dtype: int64

In [157]:
merged_client_df[merged_client_df.isnull().sum(axis=1) > 5]

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation
2486,355337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Control
7251,1037867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
8525,1227228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
11296,1618749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15523,2222915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
34329,4876926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
36220,5144725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
37164,5277910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
42470,6021001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52406,7402828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Control


In [158]:
client_df_cleaned = merged_client_df[merged_client_df.isnull().sum(axis=1) <= 5]

In [159]:
client_df_cleaned.isnull().sum()

client_id               0
clnt_tenure_yr          0
clnt_tenure_mnth        0
clnt_age                1
gendr                   0
num_accts               0
bal                     0
calls_6_mnth            0
logons_6_mnth           0
Variation           20107
dtype: int64

In [160]:
web_exp_df = pd.merge(web_df, experiment_df, on='client_id', how='right')

In [161]:
web_exp_df['Variation'].value_counts()

Variation
Test       177847
Control    143462
Name: count, dtype: int64

In [162]:
web_exp_df.shape

(449831, 6)

In [163]:
web_exp_df = web_exp_df[web_exp_df['Variation'].notnull()]

In [164]:
web_exp_df.shape

(321309, 6)

In [165]:
client_df_cleaned.describe().round(1)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,70595.0,70595.0,70595.0,70594.0,70595.0,70595.0,70595.0,70595.0
mean,5005020.9,12.1,150.7,46.4,2.3,147445.2,3.4,5.6
std,2877269.2,6.9,82.1,15.6,0.5,301508.7,2.2,2.4
min,169.0,2.0,33.0,13.5,1.0,13789.4,0.0,1.0
25%,2519604.5,6.0,82.0,32.5,2.0,37346.8,1.0,4.0
50%,5016969.0,11.0,136.0,47.0,2.0,63332.9,3.0,5.0
75%,7483063.5,16.0,192.0,59.0,2.0,137544.9,6.0,7.0
max,9999839.0,62.0,749.0,96.0,8.0,16320040.2,7.0,9.0


In [166]:
client_df_cleaned[client_df_cleaned['Variation']=='Test'].describe().round(1)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,26961.0,26961.0,26961.0,26961.0,26961.0,26961.0,26961.0,26961.0
mean,4999638.5,12.0,149.9,47.2,2.2,148962.6,3.1,6.1
std,2877509.7,6.8,81.8,15.5,0.5,307373.7,2.2,2.2
min,555.0,2.0,33.0,17.0,1.0,23789.4,0.0,3.0
25%,2516570.0,6.0,82.0,33.5,2.0,39729.2,1.0,4.0
50%,5025928.0,11.0,134.0,47.5,2.0,65468.4,3.0,6.0
75%,7470213.0,15.0,191.0,59.5,2.0,139478.6,5.0,8.0
max,9999832.0,55.0,669.0,96.0,7.0,16320040.2,6.0,9.0


In [167]:
client_df_cleaned[client_df_cleaned['Variation']=='Control'].describe().round(1)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,23527.0,23527.0,23527.0,23526.0,23527.0,23527.0,23527.0,23527.0
mean,5013661.4,12.1,151.1,47.5,2.3,150147.3,3.1,6.2
std,2877353.3,6.9,82.1,15.5,0.5,295807.0,2.2,2.2
min,1028.0,2.0,33.0,17.0,2.0,23789.6,0.0,3.0
25%,2515289.0,6.0,82.0,33.5,2.0,40123.4,1.0,4.0
50%,5023798.0,11.0,137.0,48.5,2.0,66024.2,3.0,6.0
75%,7483504.0,16.0,192.0,60.0,2.0,140606.0,5.0,8.0
max,9998921.0,55.0,669.0,96.0,7.0,8292996.2,6.0,9.0


In [168]:
client_df_cleaned[client_df_cleaned['Variation'].isnull()].describe().round(1)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,20107.0,20107.0,20107.0,20107.0,20107.0,20107.0,20107.0,20107.0
mean,5002128.1,12.1,151.3,44.2,2.3,142249.0,4.1,4.1
std,2876968.0,6.9,82.5,15.6,0.5,300124.2,2.2,2.2
min,169.0,2.0,33.0,13.5,1.0,13789.4,1.0,1.0
25%,2532094.5,6.0,82.0,30.5,2.0,30147.7,2.0,2.0
50%,4998573.0,11.0,138.0,45.0,2.0,56338.1,4.0,4.0
75%,7493638.5,16.0,193.0,56.5,2.0,130512.7,6.0,6.0
max,9999839.0,62.0,749.0,92.5,8.0,12838418.0,7.0,7.0


In [169]:
client_df_cleaned['bal_quartile'] = pd.qcut(client_df_cleaned['bal'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

C:\Users\olgeo\AppData\Local\Temp\ipykernel_24936\1511590950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df_cleaned['bal_quartile'] = pd.qcut(client_df_cleaned['bal'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])


In [170]:
client_df_cleaned.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,bal_quartile
0,169,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0,NaN,Q4
1,555,3.0,46.0,29.5,U,2.0,25454.66,2.0,6.0,Test,Q1
2,647,12.0,151.0,57.5,M,2.0,30525.80,0.0,4.0,Test,Q1
3,722,11.0,143.0,59.5,F,2.0,22466.17,1.0,1.0,NaN,Q1
4,934,9.0,109.0,51.0,F,2.0,32522.88,0.0,3.0,Test,Q1


In [171]:
client_df_cleaned[client_df_cleaned['bal_quartile']=='Q1']['gendr'].value_counts()

gendr
U    7844
F    5219
M    4583
X       3
Name: count, dtype: int64

In [172]:
client_df_cleaned[client_df_cleaned['bal_quartile']=='Q4'].describe().round(1)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,17649.0,17649.0,17649.0,17649.0,17649.0,17649.0,17649.0,17649.0
mean,4954611.4,14.9,184.7,54.6,2.5,419689.2,4.2,6.4
std,2870445.7,7.7,92.7,13.2,0.7,511980.7,2.1,2.2
min,169.0,2.0,33.0,16.0,2.0,137547.3,0.0,1.0
25%,2471025.0,8.0,105.0,45.5,2.0,182199.1,2.0,5.0
50%,4966245.0,14.0,177.0,56.0,2.0,264313.4,5.0,7.0
75%,7392799.0,21.0,253.0,64.0,3.0,451959.9,6.0,9.0
max,9999832.0,62.0,749.0,94.0,8.0,16320040.2,7.0,9.0


In [173]:
client_df_cleaned[client_df_cleaned['bal_quartile']=='Q4']['gendr'].value_counts()

gendr
M    8250
F    5707
U    3692
Name: count, dtype: int64

In [216]:
web_exp_df.sort_values(by=['client_id', 'visit_id', 'date_time']).head(30)

,client_id,visitor_id,visit_id,process_step,date_time,Variation
94261,555,402506806_56087378777,637149525_38041617439_716659,start,15/04/17 12:57:56,Test
94260,555,402506806_56087378777,637149525_38041617439_716659,step_1,15/04/17 12:58:03,Test
94259,555,402506806_56087378777,637149525_38041617439_716659,step_2,15/04/17 12:58:35,Test
94258,555,402506806_56087378777,637149525_38041617439_716659,step_3,15/04/17 13:00:14,Test
94257,555,402506806_56087378777,637149525_38041617439_716659,confirm,15/04/17 13:00:34,Test
180548,647,66758770_53988066587,40369564_40101682850_311847,start,12/04/17 15:41:28,Test
180547,647,66758770_53988066587,40369564_40101682850_311847,step_1,12/04/17 15:41:35,Test
180546,647,66758770_53988066587,40369564_40101682850_311847,step_2,12/04/17 15:41:53,Test
180545,647,66758770_53988066587,40369564_40101682850_311847,step_3,12/04/17 15:45:02,Test
180544,647,66758770_53988066587,40369564_40101682850_311847,confirm,12/04/17 15:47:45,Test


In [175]:
df_duplicates = web_exp_df[web_exp_df.duplicated(subset=['client_id', 'visitor_id', 'visit_id', 'process_step', 'Variation'], keep=False)]

df_duplicates['process_step'].value_counts()


process_step
start      62095
step_1     27811
step_2     20169
step_3     13634
confirm     8919
Name: count, dtype: int64

In [230]:
web_exp_df['date_time'] = pd.to_datetime(web_exp_df['date_time'], format='%d/%m/%y %H:%M:%S')

In [264]:
test_df = web_exp_df.copy()
test_df = test_df.sort_values(by=['client_id', 'visit_id', 'date_time'], ascending=True)

test_df['visit_final_step'] = test_df['visit_id'] != test_df['visit_id'].shift(-1)

test_df['step_duration'] = (test_df['date_time'].shift(-1) - test_df['date_time']).dt.total_seconds()
test_df.loc[test_df['visit_final_step'] == True, 'step_duration'] = np.nan

step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
next_step = {'start': 'step_1', 'step_1': 'step_2', 'step_2': 'step_3', 'step_3': 'confirm'}

test_df['next_process_step'] = test_df['process_step'].shift(-1)
test_df['step_proceeds'] = test_df.apply(
    lambda row: row['next_process_step'] == next_step.get(row['process_step'], None),
    axis=1
)
test_df.loc[test_df['visit_final_step'] == True, 'step_proceeds'] = False

test_df['step_error'] = test_df.apply(
    lambda row: row['process_step'] == next_step.get(row['next_process_step'], None),
    axis=1
)
test_df.loc[test_df['visit_final_step'] == True, 'step_error'] = False

test_df.head(8)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,visit_final_step,step_duration,next_process_step,step_proceeds,step_error
94261,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,Test,False,7.0,step_1,True,False
94260,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,Test,False,32.0,step_2,True,False
94259,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,Test,False,99.0,step_3,True,False
94258,555,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,Test,False,20.0,confirm,True,False
94257,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,Test,True,NaN,start,False,False
180548,647,66758770_53988066587,40369564_40101682850_311847,start,2017-04-12 15:41:28,Test,False,7.0,step_1,True,False
180547,647,66758770_53988066587,40369564_40101682850_311847,step_1,2017-04-12 15:41:35,Test,False,18.0,step_2,True,False
180546,647,66758770_53988066587,40369564_40101682850_311847,step_2,2017-04-12 15:41:53,Test,False,189.0,step_3,True,False


In [266]:

def step_checks(df):
    new_df = df.copy()
    new_df
    new_df = new_df.sort_values(by=['client_id', 'visit_id', 'date_time'])

    #Adds a column to say if that step is the final step of the visit
    new_df['visit_final_step'] = new_df['visit_id'] != new_df['visit_id'].shift(-1)

    #Adds a column for the duration (in seconds) of each step
    new_df['step_duration'] = (new_df['date_time'].shift(-1) - new_df['date_time']).dt.total_seconds()
    new_df.loc[new_df['visit_final_step'] == True, 'step_duration'] = np.nan

    step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
    next_step = {'start': 'step_1', 'step_1': 'step_2', 'step_2': 'step_3', 'step_3': 'confirm'}

    #Adds "next_process_step" column with the step from the following row
    new_df['next_process_step'] = new_df['process_step'].shift(-1)
    
    #Adds "step_proceeds" column which checks if the next step is the same visit and moved forward a step
    new_df['step_proceeds'] = new_df.apply(
        lambda row: row['next_process_step'] == next_step.get(row['process_step'], None),
        axis=1
    )
    new_df.loc[new_df['visit_final_step'] == True, 'step_proceeds'] = False

    #Adds "step_error" column which checks if the next step is the same visit and moved back a step
    new_df['step_error'] = new_df.apply(
        lambda row: row['process_step'] == next_step.get(row['next_process_step'], None),
        axis=1
    )
    new_df.loc[new_df['visit_final_step'] == True, 'step_error'] = False

    #Removes "next_process_step" column with the step from the following row
    new_df.drop(columns='next_process_step', inplace=True)
    
    return (new_df)

In [271]:
analysis_df = step_checks(web_exp_df)

analysis_df.head(20)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,visit_final_step,step_duration,step_proceeds,step_error
94261,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,Test,False,7.0,True,False
94260,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,Test,False,32.0,True,False
94259,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,Test,False,99.0,True,False
94258,555,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,Test,False,20.0,True,False
94257,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,Test,True,NaN,False,False
180548,647,66758770_53988066587,40369564_40101682850_311847,start,2017-04-12 15:41:28,Test,False,7.0,True,False
180547,647,66758770_53988066587,40369564_40101682850_311847,step_1,2017-04-12 15:41:35,Test,False,18.0,True,False
180546,647,66758770_53988066587,40369564_40101682850_311847,step_2,2017-04-12 15:41:53,Test,False,189.0,True,False
180545,647,66758770_53988066587,40369564_40101682850_311847,step_3,2017-04-12 15:45:02,Test,False,163.0,True,False
180544,647,66758770_53988066587,40369564_40101682850_311847,confirm,2017-04-12 15:47:45,Test,True,NaN,False,False


In [281]:
analysis_df.groupby(['Variation','process_step'])['step_error'].mean()

Variation  process_step
Control    confirm         0.007426
           start           0.000000
           step_1          0.083996
           step_2          0.053736
           step_3          0.103839
Test       confirm         0.001127
           start           0.000000
           step_1          0.165188
           step_2          0.101455
           step_3          0.088322
Name: step_error, dtype: float64

In [283]:
analysis_df.groupby(['Variation','process_step'])['step_proceeds'].mean()

Variation  process_step
Control    confirm         0.000000
           start           0.541296
           step_1          0.752663
           step_2          0.821718
           step_3          0.670498
Test       confirm         0.000000
           start           0.590750
           step_1          0.709451
           step_2          0.780858
           step_3          0.709199
Name: step_proceeds, dtype: float64

In [285]:
analysis_df.groupby(['Variation','process_step'])['visit_final_step'].mean()

Variation  process_step
Control    confirm         0.879063
           start           0.202571
           step_1          0.117972
           step_2          0.056401
           step_3          0.098750
Test       confirm         0.836016
           start           0.167334
           step_1          0.080969
           step_2          0.042822
           step_3          0.068896
Name: visit_final_step, dtype: float64

In [293]:
analysis_df[analysis_df['step_proceeds']==True].groupby(['Variation','process_step'])['step_duration'].mean()

Variation  process_step
Control    start            37.639736
           step_1           33.824004
           step_2           87.007992
           step_3          127.149176
Test       start            31.071369
           step_1           37.098713
           step_2           86.470245
           step_3          104.833315
Name: step_duration, dtype: float64